In [1]:
from sympy.physics.quantum.qubit import Qubit
from sympy import Matrix, sqrt, symbols, simplify
import sympy as sp
from IPython.display import display, Latex
from itertools import product

In [2]:
from sympy.physics.quantum import Ket, Bra, Dagger, represent, TensorProduct

In [3]:
from sympy.physics.matrices import msigma

In [4]:
import random
import math
import cmath

In [5]:
import numpy as np

In [6]:
import itertools

In [7]:
# Step 1: Symbols define karein
alpha, beta = symbols('α β')

# Step 2: Quantum state banayein (example: superposition state)
psi = alpha * Qubit('0') + beta * Qubit('1')

# Step 3: Density matrix calculate karein
rho = psi * Dagger(psi)

# Step 4: Matrix ko expand aur simplify karein
density_matrix = rho.doit().expand()
simplified_matrix = simplify(density_matrix)
simplified_matrix

α*Dagger(α)*|0>*<0| + α*Dagger(β)*|0>*<1| + β*Dagger(α)*|1>*<0| + β*Dagger(β)*|1>*<1|

In [8]:
res = represent(simplified_matrix.doit(), basis_set=[Qubit('0'), Qubit('1')])
res

# this is for one qubit, lets do for 2

Matrix([
[α*Dagger(α), α*Dagger(β)],
[β*Dagger(α), β*Dagger(β)]])

In [9]:
def get_rho_bipartite(alpha, beta, gamma, delta):
    psi = alpha * TensorProduct(Qubit(0), Qubit(0)) + beta * TensorProduct(Qubit(0), Qubit(1)) + gamma * TensorProduct(Qubit(1), Qubit(0)) + delta * TensorProduct(Qubit(1), Qubit(1))
    
    rho = psi * Dagger(psi)
    density_matrix = rho.doit().expand()
    simplified_matrix = simplify(density_matrix)
    print(simplified_matrix)
    
    rho = represent(simplified_matrix.doit(), basis_set=[Qubit('0'), Qubit('1')])
    return rho

In [10]:
dA, dB = 2, 2

In [11]:
def permute_dims(tensor, perm):
    """
    Permutes the dimensions of a sympy MutableDenseNDimArray according to the given permutation tuple.
    """
    old_shape = tensor.shape
    new_shape = tuple(old_shape[i] for i in perm)
    new_tensor = sp.MutableDenseNDimArray.zeros(*new_shape)
    for new_index in product(*[range(s) for s in new_shape]):
        # Compute the corresponding old index.
        old_index = [None] * len(new_index)
        for new_axis, old_axis in enumerate(perm):
            old_index[old_axis] = new_index[new_axis]
        new_tensor[new_index] = tensor[tuple(old_index)]
    return new_tensor


In [12]:
def compute_entanglement_measures_symbolic(rho, dA, dB, display_output=True):
    """
    Computes entanglement measures for a bipartite system (dA x dB).
    """
    N = dA * dB
    assert rho.shape[0] == N, "Product dA*dB must equal the matrix dimension."
    
    if display_output:
        display(Latex(r"Density matrix $\rho$: "))
        display(Latex(r"$$" + sp.latex(rho) + r"$$"))
    
    eig_dict_input = rho.eigenvals()
    eigenvals_input = [sp.simplify(ev) for ev, mult in eig_dict_input.items() for _ in range(mult)]
    if display_output:
        display(Latex(r"Eigenvalues of $\rho$: "))
        display(Latex(r"$$" + sp.latex(eigenvals_input) + r"$$"))
    
    # Partial Transpose
    rho_tensor = sp.MutableDenseNDimArray(rho, (dA, dB, dA, dB))
    rho_pt_tensor = permute_dims(rho_tensor, (0, 3, 2, 1))
    rho_pt = sp.Matrix(rho_pt_tensor.reshape(N, N))
    if display_output:
        display(Latex(r"Partial transpose $\rho^{T_B}$: "))
        display(Latex(r"$$" + sp.latex(rho_pt) + r"$$"))
    
    eig_dict_pt = rho_pt.eigenvals()
    eigenvals_pt = [sp.simplify(ev) for ev, mult in eig_dict_pt.items() for _ in range(mult)]
    if display_output:
        display(Latex(r"Eigenvalues of $\rho^{T_B}$: "))
        display(Latex(r"$$" + sp.latex(eigenvals_pt) + r"$$"))
    
    det_rho_pt = sp.simplify(rho_pt.det())
    if display_output:
        display(Latex(r"Determinant of $\rho^{T_B}$: "))
        display(Latex(r"$$" + sp.latex(det_rho_pt) + r"$$"))
    
    # Realignment
    rho_realigned_tensor = permute_dims(rho_tensor, (0, 2, 1, 3))
    realigned = sp.Matrix(rho_realigned_tensor.reshape(dA*dA, dB*dB))
    if display_output:
        display(Latex(r"Realigned matrix: "))
        display(Latex(r"$$" + sp.latex(realigned) + r"$$"))
    
    rhotilde_t_rhotilde = realigned.T * realigned
    if display_output:
        display(Latex(r"Symmetric for realigned: "))
        display(Latex(r"$$" + sp.latex(rhotilde_t_rhotilde) + r"$$"))
    
    # sum_sv = sp.simplify(sum(singular_vals))
    # if display_output:
    #     display(Latex(r"Sum of singular values: "))
    #     display(Latex(r"$$" + sp.latex(sum_sv) + r"$$"))
    
    # product_sv = sp.simplify(sp.Mul(*singular_vals))
    # if display_output:
    #     display(Latex(r"Product of singular values: "))
    #     display(Latex(r"$$" + sp.latex(product_sv) + r"$$"))
    
    # det_realigned = sp.simplify(realigned.det())
    # if display_output:
    #     display(Latex(r"Determinant of the realigned matrix: "))
    #     display(Latex(r"$$" + sp.latex(det_realigned) + r"$$"))
    
    # product_eigenvals = sp.simplify(sp.Mul(*eigenvals_pt))
    # if display_output:
    #     display(Latex(r"Product of eigenvalues of $\rho^{T_B}$: "))
    #     display(Latex(r"$$" + sp.latex(product_eigenvals) + r"$$"))
    
    #return eigenvals_input, eigenvals_pt, singular_vals, sum_sv, product_sv, product_eigenvals, det_rho_pt, det_realigned
    return realigned, rhotilde_t_rhotilde


In [13]:
alpha, beta, gamma, delta= symbols('α β γ δ')
realigned, rhotilde_t_rhotilde= compute_entanglement_measures_symbolic(get_rho_bipartite(alpha, beta, gamma, delta), dA, dB, True)

α*Dagger(α)*|0>x|0>*<0|x<0| + α*Dagger(β)*|0>x|0>*<0|x<1| + α*Dagger(γ)*|0>x|0>*<1|x<0| + α*Dagger(δ)*|0>x|0>*<1|x<1| + β*Dagger(α)*|0>x|1>*<0|x<0| + β*Dagger(β)*|0>x|1>*<0|x<1| + β*Dagger(γ)*|0>x|1>*<1|x<0| + β*Dagger(δ)*|0>x|1>*<1|x<1| + γ*Dagger(α)*|1>x|0>*<0|x<0| + γ*Dagger(β)*|1>x|0>*<0|x<1| + γ*Dagger(γ)*|1>x|0>*<1|x<0| + γ*Dagger(δ)*|1>x|0>*<1|x<1| + δ*Dagger(α)*|1>x|1>*<0|x<0| + δ*Dagger(β)*|1>x|1>*<0|x<1| + δ*Dagger(γ)*|1>x|1>*<1|x<0| + δ*Dagger(δ)*|1>x|1>*<1|x<1|


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [14]:
realigned

Matrix([
[α*Dagger(α), α*Dagger(β), β*Dagger(α), β*Dagger(β)],
[α*Dagger(γ), α*Dagger(δ), β*Dagger(γ), β*Dagger(δ)],
[γ*Dagger(α), γ*Dagger(β), δ*Dagger(α), δ*Dagger(β)],
[γ*Dagger(γ), γ*Dagger(δ), δ*Dagger(γ), δ*Dagger(δ)]])

In [15]:
rhotilde_t_rhotilde

Matrix([
[                            α**2*Dagger(α)**2 + α**2*Dagger(γ)**2 + γ**2*Dagger(α)**2 + γ**2*Dagger(γ)**2, α**2*Dagger(α)*Dagger(β) + α**2*Dagger(γ)*Dagger(δ) + γ**2*Dagger(α)*Dagger(β) + γ**2*Dagger(γ)*Dagger(δ),                                 α*β*Dagger(α)**2 + α*β*Dagger(γ)**2 + γ*δ*Dagger(α)**2 + γ*δ*Dagger(γ)**2,     α*β*Dagger(α)*Dagger(β) + α*β*Dagger(γ)*Dagger(δ) + γ*δ*Dagger(α)*Dagger(β) + γ*δ*Dagger(γ)*Dagger(δ)],
[α**2*Dagger(α)*Dagger(β) + α**2*Dagger(γ)*Dagger(δ) + γ**2*Dagger(α)*Dagger(β) + γ**2*Dagger(γ)*Dagger(δ),                             α**2*Dagger(β)**2 + α**2*Dagger(δ)**2 + γ**2*Dagger(β)**2 + γ**2*Dagger(δ)**2,     α*β*Dagger(α)*Dagger(β) + α*β*Dagger(γ)*Dagger(δ) + γ*δ*Dagger(α)*Dagger(β) + γ*δ*Dagger(γ)*Dagger(δ),                                 α*β*Dagger(β)**2 + α*β*Dagger(δ)**2 + γ*δ*Dagger(β)**2 + γ*δ*Dagger(δ)**2],
[                                α*β*Dagger(α)**2 + α*β*Dagger(γ)**2 + γ*δ*Dagger(α)**2 + γ*δ*Dagger(γ)**2,     α*β*Dagger(α)*Dagge

In [16]:
def generate_alpha_beta_gamma_delta():
    # Generate four random complex numbers using a Gaussian distribution.
    # Using a Gaussian can help ensure a uniform distribution on the complex sphere.
    a = random.gauss(0,1) + 1j * random.gauss(0,1)
    b = random.gauss(0,1) + 1j * random.gauss(0,1)
    c = random.gauss(0,1) + 1j * random.gauss(0,1)
    d = random.gauss(0,1) + 1j * random.gauss(0,1)
    
    # Normalize the state so that |α|^2 + |β|^2 + |γ|^2 + |δ|^2 = 1
    norm = math.sqrt(abs(a)**2 + abs(b)**2 + abs(c)**2 + abs(d)**2)
    alpha = a / norm
    beta  = b / norm
    gamma = c / norm
    delta = d / norm

    return alpha, beta, gamma, delta

In [17]:
a, b, c, d= generate_alpha_beta_gamma_delta()

In [18]:
rhotilde_t_rhotilde_subbed= simplify(rhotilde_t_rhotilde.subs({alpha: a, beta: b, gamma: c, delta: d}))

In [19]:
eig_dict= rhotilde_t_rhotilde_subbed.eigenvals()
eig_dict

{-0.219721926609201 + 0.0568510450609913*I: 1,
 -0.219721926609201 - 0.0568510450609913*I: 1,
 0.164169206920667 + 1.33703938673465e-66*I: 1,
 0.313760219249141 - 2.19253978507304e-65*I: 1}

In [20]:
def get_singular_values_from_eig_dict(eig_dict):
    singular_vals= []
    for eigenvalue, multiplicity in eig_dict.items():
        # Simplify the eigenvalue for a cleaner expression.
        eigenvalue = sp.simplify(eigenvalue)
        # print("current eig: ", eigenvalue)
        # print("multiplicity: ", multiplicity)
        # Assume eigenvalues are nonnegative; take the square root.
        s_val = sp.sqrt(eigenvalue).evalf()
        singular_vals.extend([sp.simplify(s_val)] * multiplicity)
    return singular_vals

In [21]:
singular_vals= get_singular_values_from_eig_dict(eig_dict)
singular_vals

[0.060148585125616 + 0.472588382106263*I,
 0.060148585125616 - 0.472588382106263*I,
 0.405177994121925 + 0.e-66*I,
 0.560143034634138 - 0.e-65*I]

In [22]:
sum(singular_vals)

1.0856181990073 - 0.e-65*I

In [23]:
def generate_1_normalized_real(n):
    # Generate n random real numbers from a Gaussian distribution
    random_numbers= [random.gauss(0, 1) for i in range(n)]
    
    # Compute the norm: sqrt(a^2 + b^2 + c^2 + d^2)
    sqsum= sum(map(lambda x: x**2, random_numbers))
    norm= sqrt(sqsum)
    return tuple(map(lambda x: x / norm, random_numbers))

In [24]:
rho= get_rho_bipartite(alpha, beta, gamma, delta)
rho

α*Dagger(α)*|0>x|0>*<0|x<0| + α*Dagger(β)*|0>x|0>*<0|x<1| + α*Dagger(γ)*|0>x|0>*<1|x<0| + α*Dagger(δ)*|0>x|0>*<1|x<1| + β*Dagger(α)*|0>x|1>*<0|x<0| + β*Dagger(β)*|0>x|1>*<0|x<1| + β*Dagger(γ)*|0>x|1>*<1|x<0| + β*Dagger(δ)*|0>x|1>*<1|x<1| + γ*Dagger(α)*|1>x|0>*<0|x<0| + γ*Dagger(β)*|1>x|0>*<0|x<1| + γ*Dagger(γ)*|1>x|0>*<1|x<0| + γ*Dagger(δ)*|1>x|0>*<1|x<1| + δ*Dagger(α)*|1>x|1>*<0|x<0| + δ*Dagger(β)*|1>x|1>*<0|x<1| + δ*Dagger(γ)*|1>x|1>*<1|x<0| + δ*Dagger(δ)*|1>x|1>*<1|x<1|


Matrix([
[α*Dagger(α), α*Dagger(β), α*Dagger(γ), α*Dagger(δ)],
[β*Dagger(α), β*Dagger(β), β*Dagger(γ), β*Dagger(δ)],
[γ*Dagger(α), γ*Dagger(β), γ*Dagger(γ), γ*Dagger(δ)],
[δ*Dagger(α), δ*Dagger(β), δ*Dagger(γ), δ*Dagger(δ)]])

In [25]:
def generate_realignment_scores(rhotilde_t_rhotilde, alpha, beta, gamma, delta, no_of_random_samples= 20):
    res= {}
    for i in range(no_of_random_samples):
        a, b, c, d= generate_1_normalized_real(4)
        subbed_symm= simplify(rhotilde_t_rhotilde.subs({alpha: a, beta: b, gamma: c, delta: d}))
        print("subbed symm: ", subbed_symm)
        try:
            eig_dict= subbed_symm.eigenvals()
            res[str((a, b, c, d))]= sum(get_singular_values_from_eig_dict(eig_dict))
        except e:
            print(e)
    return res
    

In [26]:
realigned, rhotilde_t_rhotilde= compute_entanglement_measures_symbolic(rho, dA, dB, False)
realigned

Matrix([
[α*Dagger(α), α*Dagger(β), β*Dagger(α), β*Dagger(β)],
[α*Dagger(γ), α*Dagger(δ), β*Dagger(γ), β*Dagger(δ)],
[γ*Dagger(α), γ*Dagger(β), δ*Dagger(α), δ*Dagger(β)],
[γ*Dagger(γ), γ*Dagger(δ), δ*Dagger(γ), δ*Dagger(δ)]])

In [27]:
rhotilde_t_rhotilde

Matrix([
[                            α**2*Dagger(α)**2 + α**2*Dagger(γ)**2 + γ**2*Dagger(α)**2 + γ**2*Dagger(γ)**2, α**2*Dagger(α)*Dagger(β) + α**2*Dagger(γ)*Dagger(δ) + γ**2*Dagger(α)*Dagger(β) + γ**2*Dagger(γ)*Dagger(δ),                                 α*β*Dagger(α)**2 + α*β*Dagger(γ)**2 + γ*δ*Dagger(α)**2 + γ*δ*Dagger(γ)**2,     α*β*Dagger(α)*Dagger(β) + α*β*Dagger(γ)*Dagger(δ) + γ*δ*Dagger(α)*Dagger(β) + γ*δ*Dagger(γ)*Dagger(δ)],
[α**2*Dagger(α)*Dagger(β) + α**2*Dagger(γ)*Dagger(δ) + γ**2*Dagger(α)*Dagger(β) + γ**2*Dagger(γ)*Dagger(δ),                             α**2*Dagger(β)**2 + α**2*Dagger(δ)**2 + γ**2*Dagger(β)**2 + γ**2*Dagger(δ)**2,     α*β*Dagger(α)*Dagger(β) + α*β*Dagger(γ)*Dagger(δ) + γ*δ*Dagger(α)*Dagger(β) + γ*δ*Dagger(γ)*Dagger(δ),                                 α*β*Dagger(β)**2 + α*β*Dagger(δ)**2 + γ*δ*Dagger(β)**2 + γ*δ*Dagger(δ)**2],
[                                α*β*Dagger(α)**2 + α*β*Dagger(γ)**2 + γ*δ*Dagger(α)**2 + γ*δ*Dagger(γ)**2,     α*β*Dagger(α)*Dagge

In [28]:
generate_realignment_scores(rhotilde_t_rhotilde, alpha, beta, gamma, delta)

subbed symm:  Matrix([[0.897106997150186, -0.0574447618965762, -0.0574447618965762, 0.00367838025992109], [-0.0574447618965762, 0.0500503272422223, 0.00367838025992109, -0.00320488987423855], [-0.0574447618965762, 0.00367838025992109, 0.0500503272422223, -0.00320488987423855], [0.00367838025992109, -0.00320488987423855, -0.00320488987423855, 0.00279234836536914]])
subbed symm:  Matrix([[0.962877275895525, -0.0392875402361195, -0.0392875402361195, 0.00160301925950964], [-0.0392875402361195, 0.0183858263841461, 0.00160301925950964, -0.000750182719983335], [-0.0392875402361195, 0.00160301925950964, 0.0183858263841461, -0.000750182719983335], [0.00160301925950964, -0.000750182719983335, -0.000750182719983335, 0.000351071336182040]])
subbed symm:  Matrix([[5.79662373175446e-7, -2.05628859376644e-5, -2.05628859376644e-5, 0.000729445790605796], [-2.05628859376644e-5, 0.000760775953075641, 0.000729445790605796, -0.0269876912339753], [-2.05628859376644e-5, 0.000729445790605796, 0.00076077595307

{'(-0.887501028648928, -0.0339701345486815, 0.399373570156444, -0.227351502230216)': 1.43068293201519,
 '(0.0752295706015858, 0.127332249846971, -0.987726487438083, 0.0502334138720104)': 1.25909694806876,
 '(-0.0275490161771428, 0.965864361944414, 0.00155155506519715, -0.257574608046411)': 1.01119467060165,
 '(-0.466215769056483, 0.535795709193153, -0.299293504246394, 0.637172828210049)': 1.27339968952248,
 '(-0.494260051004366, 0.373362583533043, -0.423760402656839, 0.660858914094988)': 1.33683976370464,
 '(0.585924676721671, -0.141001353747436, -0.106491115118987, -0.790866950789265)': 1.95680770772981,
 '(-0.738614784152404, -0.152054685498695, -0.345212107452980, 0.558709382521238)': 1.93032425680790,
 '(-0.576944074968945, 0.603331392806304, -0.290259641054555, 0.467841966466791)': 1.18959179412387,
 '(0.147625539907358, -0.130879406716477, -0.895957026444848, -0.397917438206358)': 1.35201020143665,
 '(0.879963622720560, -0.306402730103375, -0.345051425890701, -0.112786981360882)'

In [29]:
# Bell state (|00⟩ + |11⟩)/√2
bell_state = (TensorProduct(Qubit('0'), Qubit('0')) + 
             TensorProduct(Qubit('1'), Qubit('1')))/sqrt(2)

# Density matrix
bell_rho = bell_state * Dagger(bell_state)
bell_rho

(|0>x|0> + |1>x|1>)*(<0|x<0| + <1|x<1|)/2

In [30]:
matrix_form = represent(bell_rho.doit(), basis_set=[Qubit('0'), Qubit('1')])

# Simplify करें और output को Matrix के रूप में प्राप्त करें
final_matrix = Matrix(simplify(matrix_form)).expand()
final_matrix

Matrix([
[1/2, 0, 0, 1/2],
[  0, 0, 0,   0],
[  0, 0, 0,   0],
[1/2, 0, 0, 1/2]])

In [31]:
# Bell state vector (|00⟩ + |11⟩)/√2 को Matrix के रूप में परिभाषित करें
bell_vector = Matrix([[1], [0], [0], [1]])/sqrt(2)

# Density matrix बनाएं (outer product)
density_matrix = bell_vector * Dagger(bell_vector)

# मैट्रिक्स को simplify करें और expand करें
rho = density_matrix.expand()
rho

Matrix([
[1/2, 0, 0, 1/2],
[  0, 0, 0,   0],
[  0, 0, 0,   0],
[1/2, 0, 0, 1/2]])

In [32]:
# Execute the optimized function

result = compute_entanglement_measures_symbolic(rho, dA, dB)
result

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

(Matrix([
 [1/2,   0,   0,   0],
 [  0, 1/2,   0,   0],
 [  0,   0, 1/2,   0],
 [  0,   0,   0, 1/2]]),
 Matrix([
 [1/4,   0,   0,   0],
 [  0, 1/4,   0,   0],
 [  0,   0, 1/4,   0],
 [  0,   0,   0, 1/4]]))

In [33]:
ajeeb_matrix= sp.Matrix([
    [0.5, 0, 0, 0],
    [0, 0, 0.5, 0],
    [0, 0.5, 0, 0],
    [0, 0, 0, 0.5]
])
ajeeb_matrix


Matrix([
[0.5,   0,   0,   0],
[  0,   0, 0.5,   0],
[  0, 0.5,   0,   0],
[  0,   0,   0, 0.5]])

In [34]:
ajeeb_matrix.det()

-0.0625000000000000

In [35]:
1 / 16

0.0625

In [36]:
I_2= Matrix.eye(2)
rx, ry, rz, sx, sy, sz = symbols('rx ry rz sx sy sz')
rho

Matrix([
[1/2, 0, 0, 1/2],
[  0, 0, 0,   0],
[  0, 0, 0,   0],
[1/2, 0, 0, 1/2]])

In [37]:
rho.trace()

1

In [38]:
t_nm = lambda n, m: (rho * TensorProduct(msigma(n), msigma(m))).trace()

In [39]:
T= Matrix([[t_nm(n, m) for m in range(1, 4)] for n in range(1, 4)])

In [40]:
display(T)

Matrix([
[1,  0, 0],
[0, -1, 0],
[0,  0, 1]])

In [41]:
bchsh_sum_correlations= Matrix.zeros(4)
for n in range(1, 4):
    for m in range(1, 4):
        tba= T[n - 1, m - 1] * TensorProduct(msigma(n), msigma(m))
        bchsh_sum_correlations += tba
    # print("sum at the end of n= ", n, "is: ", s)

In [42]:
bchsh_sum_correlations

Matrix([
[1,  0,  0, 2],
[0, -1,  0, 0],
[0,  0, -1, 0],
[2,  0,  0, 1]])

In [43]:
s= Matrix([sx, sy, sz])
r= Matrix([rx, ry, rz])

In [44]:
ssigma= Matrix.zeros(2)
rsigma= Matrix.zeros(2)
for i in range(1, 4):
    ssigma += s[i-1] * msigma(i)
    rsigma += r[i-1] * msigma(i)

In [45]:
rho_bell_basis= Matrix.zeros(4)

In [46]:
rho_bell_basis+= TensorProduct(I_2, I_2)
rho_bell_basis

Matrix([
[1, 0, 0, 0],
[0, 1, 0, 0],
[0, 0, 1, 0],
[0, 0, 0, 1]])

In [47]:
res= TensorProduct(rsigma, I_2)
rho_bell_basis += res
res

Matrix([
[       rz,         0, rx - I*ry,         0],
[        0,        rz,         0, rx - I*ry],
[rx + I*ry,         0,       -rz,         0],
[        0, rx + I*ry,         0,       -rz]])

In [48]:
res=  TensorProduct(I_2, ssigma)
rho_bell_basis += res
res

Matrix([
[       sz, sx - I*sy,         0,         0],
[sx + I*sy,       -sz,         0,         0],
[        0,         0,        sz, sx - I*sy],
[        0,         0, sx + I*sy,       -sz]])

In [49]:
rho_bell_basis += bchsh_sum_correlations
bchsh_sum_correlations

Matrix([
[1,  0,  0, 2],
[0, -1,  0, 0],
[0,  0, -1, 0],
[2,  0,  0, 1]])

In [50]:
rho_bell_basis

Matrix([
[rz + sz + 2, sx - I*sy, rx - I*ry,            2],
[  sx + I*sy,   rz - sz,         0,    rx - I*ry],
[  rx + I*ry,         0,  -rz + sz,    sx - I*sy],
[          2, rx + I*ry, sx + I*sy, -rz - sz + 2]])

In [51]:
eig_dict= rho_bell_basis.eigenvals()

In [52]:
eig_dict

{Piecewise((-sqrt(4*rx**2/3 + 4*ry**2/3 + 4*rz**2/3 + 4*sx**2/3 + 4*sy**2/3 + 4*sz**2/3 - 2*(-(-8*rx*sx + 8*ry*sy - 8*rz*sz - 8)**2/8 - (-2*rx**2 - 2*ry**2 - 2*rz**2 - 2*sx**2 - 2*sy**2 - 2*sz**2 - 6)**3/108 + (-2*rx**2 - 2*ry**2 - 2*rz**2 - 2*sx**2 - 2*sy**2 - 2*sz**2 - 6)*(rx**4 + 2*rx**2*ry**2 + 2*rx**2*rz**2 - 2*rx**2*sx**2 - 2*rx**2*sy**2 - 2*rx**2*sz**2 + 2*rx**2 - 8*rx*sx + ry**4 + 2*ry**2*rz**2 - 2*ry**2*sx**2 - 2*ry**2*sy**2 - 2*ry**2*sz**2 + 2*ry**2 + 8*ry*sy + rz**4 - 2*rz**2*sx**2 - 2*rz**2*sy**2 - 2*rz**2*sz**2 + 2*rz**2 - 8*rz*sz + sx**4 + 2*sx**2*sy**2 + 2*sx**2*sz**2 + 2*sx**2 + sy**4 + 2*sy**2*sz**2 + 2*sy**2 + sz**4 + 2*sz**2 - 3)/3)**(1/3) + 4)/2 + sqrt(8*rx**2/3 + 8*ry**2/3 + 8*rz**2/3 + 8*sx**2/3 + 8*sy**2/3 + 8*sz**2/3 + 2*(-(-8*rx*sx + 8*ry*sy - 8*rz*sz - 8)**2/8 - (-2*rx**2 - 2*ry**2 - 2*rz**2 - 2*sx**2 - 2*sy**2 - 2*sz**2 - 6)**3/108 + (-2*rx**2 - 2*ry**2 - 2*rz**2 - 2*sx**2 - 2*sy**2 - 2*sz**2 - 6)*(rx**4 + 2*rx**2*ry**2 + 2*rx**2*rz**2 - 2*rx**2*sx**2 - 2*rx*

In [53]:
rho_tensor = sp.MutableDenseNDimArray(rho_bell_basis, (dA, dB, dA, dB))
rho_realigned_tensor = permute_dims(rho_tensor, (0, 2, 1, 3))
realigned = sp.Matrix(rho_realigned_tensor.reshape(dA*dA, dB*dB))
rhotilde_t_rhotilde= realigned.T * realigned
rhotilde_t_rhotilde

Matrix([
[                   (rx - I*ry)**2 + (rx + I*ry)**2 + (-rz + sz)**2 + (rz + sz + 2)**2, 2*rx - 2*I*ry + (-rz + sz)*(sx - I*sy) + (sx - I*sy)*(rz + sz + 2), 2*rx + 2*I*ry + (-rz + sz)*(sx + I*sy) + (sx + I*sy)*(rz + sz + 2), (rx - I*ry)**2 + (rx + I*ry)**2 + (-rz + sz)*(-rz - sz + 2) + (rz - sz)*(rz + sz + 2)],
[                   2*rx - 2*I*ry + (-rz + sz)*(sx - I*sy) + (sx - I*sy)*(rz + sz + 2),                                               2*(sx - I*sy)**2 + 4,                                          2*(sx - I*sy)*(sx + I*sy),                    2*rx - 2*I*ry + (rz - sz)*(sx - I*sy) + (sx - I*sy)*(-rz - sz + 2)],
[                   2*rx + 2*I*ry + (-rz + sz)*(sx + I*sy) + (sx + I*sy)*(rz + sz + 2),                                          2*(sx - I*sy)*(sx + I*sy),                                               2*(sx + I*sy)**2 + 4,                    2*rx + 2*I*ry + (rz - sz)*(sx + I*sy) + (sx + I*sy)*(-rz - sz + 2)],
[(rx - I*ry)**2 + (rx + I*ry)**2 + (-rz + sz)*(-rz - sz

In [54]:
generate_1_normalized_real(6)

(-0.106651517260556,
 0.532563140325097,
 0.357355496351069,
 -0.674252804129866,
 -0.311294594664216,
 -0.160554776062962)

In [55]:
def generate_realignment_scores_bell_basis(rhotilde_t_rhotilde, rx, ry, rz, sx, sy, sz, no_of_random_samples= 20):
    res= {}
    for i in range(no_of_random_samples):
        a, b, c, d, e, f = generate_1_normalized_real(6)
        subbed_symm= simplify(rhotilde_t_rhotilde.subs({rx: a, ry: b, rz: c, sx: d, sy: e, sz: f}))
        print("subbed symm: ", subbed_symm)
        try:
            eig_dict= subbed_symm.eigenvals()
            res[str((a, b, c, d))]= sum(get_singular_values_from_eig_dict(eig_dict))
        except e:
            print(e)
    return res

In [56]:
generate_realignment_scores_bell_basis(rhotilde_t_rhotilde, rx, ry, rz, sx, sy, sz)

subbed symm:  Matrix([[3.34626092678612, -0.592173310172567 - 0.333613548387647*I, -0.592173310172567 + 0.333613548387647*I, -0.329040398794266], [-0.592173310172567 - 0.333613548387647*I, 5.1305177169992 - 0.813197750053435*I, 1.39260938139202, -1.70762293256017 + 0.0258931620756989*I], [-0.592173310172567 + 0.333613548387647*I, 1.39260938139202, 5.1305177169992 + 0.813197750053435*I, -1.70762293256017 - 0.0258931620756989*I], [-0.329040398794266, -1.70762293256017 + 0.0258931620756989*I, -1.70762293256017 - 0.0258931620756989*I, 4.98191684411203]])
subbed symm:  Matrix([[4.84246439918377, -0.95247976037569 - 1.06949899623149*I, -0.95247976037569 + 1.06949899623149*I, 0.166092048284938], [-0.95247976037569 - 1.06949899623149*I, 3.26025052038375 + 1.35767353549738*I, 1.54612636016028, -1.43300312710082 - 1.87854235291933*I], [-0.95247976037569 + 1.06949899623149*I, 1.54612636016028, 3.26025052038375 - 1.35767353549738*I, -1.43300312710082 + 1.87854235291933*I], [0.166092048284938, -1.4

{'(0.219268651658843, 0.332904540806710, 0.146658833191672, -0.794217712342028)': 8.43491068052467,
 '(-0.147378271892507, -0.0189453227507205, 0.365063100032232, -0.448992449976620)': 7.93906327017170,
 '(0.0257255860976131, -0.225977466313379, 0.301306822478718, -0.562384116192942)': 7.85485865731066 - 0.e-63*I,
 '(-0.153747027691418, -0.798439296999759, -0.0176397900041277, 0.512801024156750)': 7.98161138821280,
 '(0.406907919091983, -0.178937578971443, 0.729552842495088, -0.442876179510025)': 8.73135497107899 + 0.e-65*I,
 '(0.213672983501562, 0.0236322313771547, 0.407030302404499, 0.712986784629745)': 8.06396873007625 + 0.e-66*I,
 '(0.478579411081613, -0.0162812089506051, -0.350553054797578, 0.409388254420636)': 7.87302134293638 + 0.e-62*I,
 '(0.384731183366232, 0.0344919786051264, -0.499136748787901, -0.441006485105292)': 8.86784111638018,
 '(-0.857997821050844, 0.142162608149696, 0.461354364471139, -0.128895682474093)': 8.40423073251373 + 0.e-64*I,
 '(-0.0487767083803278, 0.40454